# Pandasの参照と代入 

In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
df = pd.DataFrame([["A",2],["B",1],["C",4],["A",2],["E",5]], columns=["alpha","beta"])
df

,alpha,beta
0,A,2
1,B,1
2,C,4
3,A,2
4,E,5


## DataFrameのインデキシングを利用して参照

DataFrameのインデキシングはカラム名で行う．指定した列のSeriesがビューとして返る

In [4]:
df["alpha"]

0    A
1    B
2    C
3    A
4    E
Name: alpha, dtype: object

比較を行うとboolのSeriesで返る

In [5]:
df["alpha"] == "A" 

0     True
1    False
2    False
3     True
4    False
Name: alpha, dtype: bool

ndarrayのように，fancy indexingを利用できる．以下の場合はブーリアンマスクを用いている．以下の例では行全てのビューが得られる．

In [6]:
df[df["alpha"] == "A"]

,alpha,beta
0,A,2
3,A,2


インデキシングを用いて代入を行うことはできない．以下の例では，alphaカラムが"A"の行の"beta"カラムに，10を代入しようとしている．

In [7]:
df[df["alpha"] == "A"]["beta"] = 10
df

C:\Users\Asami Naoto\Anaconda3\envs\torch_py37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,alpha,beta
0,A,2
1,B,1
2,C,4
3,A,2
4,E,5


## pd.DataFrame.locを利用して参照，代入

locメソッドは，行による情報と列による情報を指定する．通常のnumpyやリストと同様に，一番最初が行で，次が列の情報にあたる．

In [8]:
df.loc[0,"beta"]

2

以下では，先ほどのインデキシングのように，特定のカラムをSeriesとして取得している．

In [9]:
df.loc[:,"alpha"]

0    A
1    B
2    C
3    A
4    E
Name: alpha, dtype: object

locを使えば，代入を行うことができる．また，行・列の情報はboolのSeriesでもよい．

In [10]:
df.loc[df["alpha"]=="A","beta"] = 10
df

,alpha,beta
0,A,10
1,B,1
2,C,4
3,A,10
4,E,5


### 部分Seriesを代入 

pd.Seriesを代入すると，うまく代入できず，NaNが出てしまう．おそらくindexの関係？

In [11]:
df.loc[df["alpha"]=="A","beta"] = pd.Series([100,200])
df

,alpha,beta
0,A,100.0
1,B,1.0
2,C,4.0
3,A,NaN
4,E,5.0


一方ndarrayやリストは代入できる

In [12]:
df.loc[df["alpha"]=="A","beta"] = np.array([100,200])
df

,alpha,beta
0,A,100.0
1,B,1.0
2,C,4.0
3,A,200.0
4,E,5.0


In [13]:
df.loc[df["alpha"]=="A","beta"] = [1000,2000]
df

,alpha,beta
0,A,1000.0
1,B,1.0
2,C,4.0
3,A,2000.0
4,E,5.0


In [14]:
df.loc[:,"beta"] = np.array([2,1,4,2,5])
df

,alpha,beta
0,A,2
1,B,1
2,C,4
3,A,2
4,E,5


### 部分DataFrameを 代入

ndarrayやリストを用いて代入できる．その場合，順番を正しく並び替え，一致したサイズである必要がある．

In [15]:
df.loc[df["alpha"]=="A"] = np.array([["E",100],["G",200]])
df

,alpha,beta
0,E,100
1,B,1
2,C,4
3,G,200
4,E,5


### DataFrameをupsert 

sqlのupsertのように，同じインデックス・カラム構造を持つpd.DataFrameを別のpd,DataFrameにupsert(存在するものはupdate,存在しないものはinsert)するメソッドは存在しない．そこで元のpd.DataFrameのうち新しいpd.DataFrameに存在しない部分にあたらしいpd.DataFrameを結合させる．

In [16]:
part_df = pd.DataFrame({"alpha":["E","B", "F"],"beta":[20, 10, 30]}, index=[4,1,5])
notin_bool = ~df.index.isin(part_df.index)
df = df.loc[notin_bool].append(part_df)
df.sort_index(inplace=True)
df

,alpha,beta
0,E,100
1,B,10
2,C,4
3,G,200
4,E,20
5,F,30


元のpd.DataFrameのサイズが大きい場合，大きいサイズでisinをとることになるため代替が必要になるかもしれない．

## 利用できる比較 

& や ==, < などに加えて，inも利用できる．

In [17]:
df["alpha"].isin(["E","B"])

0     True
1     True
2    False
3    False
4     True
5    False
Name: alpha, dtype: bool

In [18]:
df.loc[df["alpha"].isin(["E","B"]),"beta"] = 1000
df

,alpha,beta
0,E,1000
1,B,1000
2,C,4
3,G,200
4,E,1000
5,F,30
